#### Przygotowanie ramki danych ze statystykami z ostatnich meczów

In [5]:
rolling_stats = matches[[col for col in matches.columns if 'last' in col or 'over_2.5' in col]]

In [6]:
rolling_stats.groupby(['over_2.5']).mean()

,home_last5_matches_played,home_last5_wins,home_last5_draws,home_last5_defeats,home_last5_goal_difference,home_last5_goals_conceded,home_last5_minutes,home_last5_goals,home_last5_assists,home_last5_pens_made,home_last5_pens_att,home_last5_shots,home_last5_shots_on_target,home_last5_cards_yellow,home_last5_cards_red,home_last5_touches,home_last5_tackles,home_last5_interceptions,home_last5_blocks,home_last5_xg,home_last5_npxg,home_last5_xg_assist,home_last5_sca,home_last5_gca,home_last5_passes_completed,home_last5_passes,home_last5_passes_pct,home_last5_progressive_passes,home_last5_carries,home_last5_progressive_carries,home_last5_take_ons,home_last5_take_ons_won,home_last5_passes_total_distance,home_last5_passes_progressive_distance,home_last5_passes_completed_short,home_last5_passes_short,home_last5_passes_pct_short,home_last5_passes_completed_medium,home_last5_passes_medium,home_last5_passes_pct_medium,home_last5_passes_completed_long,home_last5_passes_long,home_last5_passes_pct_long,home_last5_pass_xa,home_last5_assisted_shots,home_last5_passes_into_final_third,home_last5_passes_into_penalty_area,home_last5_crosses_into_penalty_area,home_last5_passes_live,home_last5_passes_dead,home_last5_passes_free_kicks,home_last5_through_balls,home_last5_passes_switches,home_last5_crosses,home_last5_throw_ins,home_last5_corner_kicks,home_last5_corner_kicks_in,home_last5_corner_kicks_out,home_last5_corner_kicks_straight,home_last5_passes_offsides,home_last5_passes_blocked,home_last5_tackles_won,home_last5_tackles_def_3rd,home_last5_tackles_mid_3rd,home_last5_tackles_att_3rd,home_last5_challenge_tackles,home_last5_challenges,home_last5_challenge_tackles_pct,home_last5_challenges_lost,home_last5_blocked_shots,home_last5_blocked_passes,home_last5_tackles_interceptions,home_last5_clearances,home_last5_errors,home_last5_touches_def_pen_area,home_last5_touches_def_3rd,home_last5_touches_mid_3rd,home_last5_touches_att_3rd,home_last5_touches_att_pen_area,home_last5_touches_live_ball,home_last5_take_ons_won_pct,home_last5_take_ons_tackled,home_last5_take_ons_tackled_pct,home_last5_carries_distance,home_last5_carries_progressive_distance,home_last5_carries_into_final_third,home_last5_carries_into_penalty_area,home_last5_miscontrols,home_last5_dispossessed,home_last5_passes_received,home_last5_progressive_passes_received,home_last5_cards_yellow_red,home_last5_fouls,home_last5_fouled,home_last5_offsides,home_last5_pens_won,home_last5_pens_conceded,home_last5_own_goals,home_last5_ball_recoveries,home_last5_aerials_won,home_last5_aerials_lost,home_last5_aerials_won_pct,home_last5_points,away_last5_matches_played,away_last5_wins,away_last5_draws,away_last5_defeats,away_last5_goal_difference,away_last5_goals_conceded,away_last5_minutes,away_last5_goals,away_last5_assists,away_last5_pens_made,away_last5_pens_att,away_last5_shots,away_last5_shots_on_target,away_last5_cards_yellow,away_last5_cards_red,away_last5_touches,away_last5_tackles,away_last5_interceptions,away_last5_blocks,away_last5_xg,away_last5_npxg,away_last5_xg_assist,away_last5_sca,away_last5_gca,away_last5_passes_completed,away_last5_passes,away_last5_passes_pct,away_last5_progressive_passes,away_last5_carries,away_last5_progressive_carries,away_last5_take_ons,away_last5_take_ons_won,away_last5_passes_total_distance,away_last5_passes_progressive_distance,away_last5_passes_completed_short,away_last5_passes_short,away_last5_passes_pct_short,away_last5_passes_completed_medium,away_last5_passes_medium,away_last5_passes_pct_medium,away_last5_passes_completed_long,away_last5_passes_long,away_last5_passes_pct_long,away_last5_pass_xa,away_last5_assisted_shots,away_last5_passes_into_final_third,away_last5_passes_into_penalty_area,away_last5_crosses_into_penalty_area,away_last5_passes_live,away_last5_passes_dead,away_last5_passes_free_kicks,away_last5_through_balls,away_last5_passes_switches,away_last5_crosses,away_last5_throw_ins,away_last5_corner_kicks,away_last5_corner_kicks_in,away_last5_corner_ki

In [8]:
X = rolling_stats.drop(columns=["over_2.5"])  # Features
y = rolling_stats["over_2.5"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
log_reg = LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr')
log_reg.fit(X_train, y_train)
model = SelectFromModel(log_reg, prefit=True)
selected_features = X.columns[model.get_support()]
print("Selected Features based on Logistic Regression:")
print(selected_features)
X_selected = model.transform(X_scaled)

Selected Features based on Logistic Regression:
Index(['home_last5_wins', 'home_last5_defeats', 'home_last5_goals_conceded',
       'home_last5_minutes', 'home_last5_goals', 'home_last5_pens_made',
       'home_last5_pens_att', 'home_last5_shots', 'home_last5_shots_on_target',
       'home_last5_cards_yellow',
       ...
       'away_last30_cards_yellow_red', 'away_last30_fouls',
       'away_last30_fouled', 'away_last30_offsides', 'away_last30_pens_won',
       'away_last30_pens_conceded', 'away_last30_own_goals',
       'away_last30_ball_recoveries', 'away_last30_aerials_won_pct',
       'away_last30_points'],
      dtype='object', length=270)


In [9]:
rolling_stats =  pd.DataFrame(X_selected, columns=selected_features)
rolling_stats["over_2.5"] = y.reset_index(drop=True)

#### Wczytywanie danych do torcha

In [10]:
class FootballDataset(Dataset):
    def __init__(self, agg_stats):
        self.agg_stats = agg_stats.drop('over_2.5', axis=1).values  # Convert to numpy array
        self.labels = agg_stats['over_2.5'].values  # Convert to numpy array
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Return features and labels as tensors
        features = torch.tensor(self.agg_stats[idx], dtype=torch.float32)
        labels = torch.tensor(self.labels[idx], dtype=torch.long)
        return features, labels

In [19]:
train_df, test_df = train_test_split(rolling_stats, test_size=0.3, random_state=42)
dataset = FootballDataset(train_df)
test_dataset = FootballDataset(test_df)
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False)

##### Customowa loss function - raczej się nie nadaje, ale można opisać próby jakby co póżniej

In [12]:
# def distance_penalty_loss(predictions, targets, penalty_factor=0.5):
#     # Apply softmax to get probabilities
#     predictions_probs = F.softmax(predictions, dim=1)

#     # Get the predicted class index (most confident prediction)
#     predicted_class = torch.argmax(predictions_probs, dim=1)
    
#     # Calculate the cross-entropy loss as usual
#     cross_entropy_loss = F.cross_entropy(predictions, targets, reduction='none')

#     # Calculate the distance between predicted class and true class
#     distance = (predicted_class != targets).float()
    
#     max_values, _ = torch.max(predictions_probs, dim=1)

#     # Apply a penalty based on the distance (e.g., penalty factor of 2.0)
#     penalty = 1 + penalty_factor * distance

#     # Return the mean penalized loss over the batch
#     penalized_loss = cross_entropy_loss * penalty * (max_values + 3)
#     return penalized_loss.mean()

#### Prosta sieć neuronowa z 3 warstwami

In [26]:
class FootballMatchPredictor(nn.Module):
    def __init__(self, input_agg_stats_size, hidden_size=64):
        super(FootballMatchPredictor, self).__init__()
        
        # Layer for aggregated statistics
        self.agg_stats_fc = nn.Sequential(
            nn.Linear(input_agg_stats_size, hidden_size),
            # nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU()
        )
        
        combined_input_size = hidden_size // 2
        self.fc_combined = nn.Sequential(
            nn.Linear(combined_input_size, hidden_size // 4),
            nn.ReLU(),
            nn.Linear(hidden_size // 4, 2),  # Output: 2 probabilities (over/under)
        )

    def forward(self, agg_stats):

        agg_stats_out = self.agg_stats_fc(agg_stats)

        output = self.fc_combined(agg_stats_out)
        
        # Apply softmax to get probabilities
        return F.softmax(output, dim=1)

#### Narazie najlepsza loss function - która gwarantuje, że prawdopodbieństwo remisów nie jest zbyt małe jak w przypadku innych loss functions

In [15]:
def weighted_cross_entropy_loss(predictions, targets):
    smoothing = 0.3
    weight_matrix = torch.tensor([
    [1.0, 5.0, 5.0],  # True label is draw (heavier penalty for home/away win predictions)
    [1.0, 1.0, 2.0],  # True label is home win
    [1.0, 2.0, 1.0]   # True label is away win
    ])
    # Apply softmax to get probabilities
    predictions_probs = F.softmax(predictions, dim=1)
    
    # One-hot encode the target labels (0 = draw, 1 = home win, 2 = away win)
    one_hot_targets = F.one_hot(targets, num_classes=3).float()

    smoothed_targets = one_hot_targets.clone()
    for i, target in enumerate(targets):
        if target == 0:  # If the true class is 'draw'
            smoothed_targets[i] = torch.tensor([0.8, 0.1, 0.1])  # 0.8 for draw, 0.1 for home/away win
        elif target == 1:  # If the true class is 'home win'
            smoothed_targets[i] = torch.tensor([0.3, 0.65, 0.05])
        elif target == 2:  # If the true class is 'away win'
            smoothed_targets[i] = torch.tensor([0.3, 0.05, 0.65])
    

    # Calculate the cross-entropy loss for each class
    loss_per_class = -smoothed_targets * torch.log(predictions_probs)
    # Weight the loss based on the distance between predicted and true class
    # For each true class, apply the corresponding row of the weight matrix
    weights = weight_matrix[targets]  # Get the corresponding weight row for each true label
    weighted_loss = loss_per_class * weights

    # Return the mean weighted loss over the batch
    return weighted_loss.sum(dim=1).mean()


In [46]:
model = FootballMatchPredictor(270)

# Loss function and optimizer
# criterion = weighted_cross_entropy_loss  # For multi-class classification
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001)


# Training loop
num_epochs = 10 
batch_size = 16

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for batch_idx, (inputs, targets) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")):


        # # Move data to device (GPU/CPU)
        # inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
        # targets = targets.to('cuda') if torch.cuda.is_available() else targets
        
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        
        # Optimize weights
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

    # Print average loss for the epoch
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Save the model
torch.save(model.state_dict(), 'football_match_predictor.pth')

Epoch 1/10: 100%|██████████| 326/326 [00:00<00:00, 447.41it/s]


Epoch [1/10], Loss: 0.6903


Epoch 2/10: 100%|██████████| 326/326 [00:00<00:00, 399.48it/s]


Epoch [2/10], Loss: 0.6876


Epoch 3/10: 100%|██████████| 326/326 [00:00<00:00, 418.11it/s]


...

Epoch [8/10], Loss: 0.6404


Epoch 9/10: 100%|██████████| 326/326 [00:00<00:00, 466.05it/s]


Epoch [9/10], Loss: 0.6246


Epoch 10/10: 100%|██████████| 326/326 [00:00<00:00, 473.20it/s]

Epoch [10/10], Loss: 0.6141


In [47]:
model.eval()

# To make predictions, you can create a DataLoader for your new data or use existing data.
# For this example, let's say we want to make predictions on the validation dataset.
# If you have a specific dataset or inputs, make sure it's in the correct format.

# Here we assume you want to predict on a subset of your training dataset for demonstration.
# If you're predicting on new data, load that data into a DataLoader accordingly.

with torch.no_grad():  # Disable gradient calculation for predictions
    predictions = []
    probabilities=[]
    actuals = []  # To store actual outcomes if you have them for comparison

    for inputs, targets in test_dataloader:  # You can replace dataloader with your validation/test DataLoader
        
        # Get model outputs
        outputs = model(inputs)  # Shape will be [batch_size, 3]
        
        # Get the predicted class indices (argmax to get the class with highest probability)
        _, predicted_classes = torch.max(outputs, dim=1)
        
        # Store predictions and actuals
        probabilities.extend(outputs)
        predictions.extend(predicted_classes.cpu().numpy())  # Convert to numpy for easy handling
        actuals.extend(targets.numpy())  # Store actual targets

# Now predictions contains the predicted class indices for the input data
# and actuals contains the true class labels (if available).

# Optionally, print the results
print("Predictions:", probabilities)
print("Predictions:", predictions)
print("Actuals:    ", actuals)

# You can calculate accuracy or other metrics if you have actual labels
accuracy = (np.array(predictions) == np.array(actuals)).mean()
print(f'Accuracy: {accuracy:.4f}')

Predictions: [tensor([0.5801, 0.4199]), tensor([0.6468, 0.3532]), tensor([0.1198, 0.8802]), tensor([0.5850, 0.4150]), tensor([0.1226, 0.8774]), tensor([0.2110, 0.7890]), tensor([0.6835, 0.3165]), tensor([0.7512, 0.2488]), tensor([0.7742, 0.2258]), tensor([0.6685, 0.3315]), tensor([0.1745, 0.8255]), tensor([0.7294, 0.2706]), tensor([0.1237, 0.8763]), tensor([0.7716, 0.2284]), tensor([0.2472, 0.7528]), tensor([0.4908, 0.5092]), tensor([0.7772, 0.2228]), tensor([0.4825, 0.5175]), tensor([0.1360, 0.8640]), tensor([0.4144, 0.5856]), tensor([0.4740, 0.5260]), tensor([0.6214, 0.3786]), tensor([0.0505, 0.9495]), tensor([0.4898, 0.5102]), tensor([0.1626, 0.8374]), tensor([0.4211, 0.5789]), tensor([0.7449, 0.2551]), tensor([0.4521, 0.5479]), tensor([0.1069, 0.8931]), tensor([0.3582, 0.6418]), tensor([0.3988, 0.6012]), tensor([0.4250, 0.5750]), tensor([0.2607, 0.7393]), tensor([0.5674, 0.4326]), tensor([0.3963, 0.6037]), tensor([0.7782, 0.2218]), tensor([0.0887, 0.9113]), tensor([0.4003, 0.5997])

In [42]:
model.eval()

# Create a DataLoader for the validation data
validation_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False)

# Function to calculate model accuracy
def calculate_accuracy(loader):
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for inputs, targets in loader:
            # inputs = inputs.to(device)
            outputs = model(inputs)
            predicted_classes = torch.argmax(outputs, dim=1)
            total_correct += (predicted_classes.cpu().numpy() == targets.numpy()).sum()
            total_samples += targets.size(0)
    
    return total_correct / total_samples

# Calculate the accuracy of the original model
original_accuracy = calculate_accuracy(validation_loader)

print(f'Original Accuracy: {original_accuracy:.4f}')

# Initialize a list to hold importances
importances = []

# Calculate importance for each feature
num_features = dataset.agg_stats.shape[1]

for i in range(num_features):  # Loop over each feature
    # Store the original values of the feature
    original_values = dataset.agg_stats[:, i].copy()

    # Shuffle the feature values
    dataset.agg_stats[:, i] = np.random.permutation(dataset.agg_stats[:, i])

    # Calculate accuracy on the shuffled dataset
    shuffled_accuracy = calculate_accuracy(validation_loader)

    # Restore the original feature values
    dataset.agg_stats[:, i] = original_values

    # Calculate the importance (drop in accuracy)
    importance = original_accuracy - shuffled_accuracy
    importances.append(importance)

# Convert importances to a numpy array for easier handling
feature_importance = np.array(importances)


# Sort features by importance
sorted_idx = np.argsort(feature_importance)[::-1]
sorted_importance = feature_importance[sorted_idx]

feature_names = df_last5.columns.drop('outcome').tolist()

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_importance)), sorted_importance, align='center')
plt.yticks(range(len(sorted_importance)), np.array(feature_names)[sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Permutation Feature Importance')
plt.show()

Original Accuracy: 0.5714


KeyboardInterrupt: 